In [2]:
import pandas as pd
import urllib.request
# Read an excel file into a variable named df
df = pd.read_excel('24996-store-overview.xlsx')
df.head()


,Location,Latitude,Longitude,Population,Url,Costs Year 2014,Costs Year 2015,Costs Year 2016,Costs Year 2017,Costs Year 2018,Costs Year 2019,Costs Year 2020,Costs Year 2021,Costs Year 2022
0,Alingsås,57.930020,12.536211,27501,https://cda.hhs.se/toystorey/24996-alings%C3%A...,15900,17100,16600,16900,16400,16900,17000,17300,15300
1,Boden,65.825119,21.688703,16848,https://cda.hhs.se/toystorey/24996-boden.xlsx,15200,16300,17100,17500,18000,18200,18500,18600,16600
2,Gävle,60.674880,17.141273,77699,https://cda.hhs.se/toystorey/24996-g%C3%A4vle....,16400,17500,18200,18100,18200,18700,19000,19100,17200
3,Göteborg,57.708870,11.974560,600473,https://cda.hhs.se/toystorey/24996-g%C3%B6tebo...,21300,22000,21400,21800,21900,22100,22200,21900,20700
4,Helsingborg,56.046467,12.694512,112496,https://cda.hhs.se/toystorey/24996-helsingborg...,18300,19100,19100,18900,19000,18900,18700,19100,16900


In [158]:
# Step 1 – Downloading data files

# iterate over every row in the DataFrame
for index, row in df.iterrows():
    urllib.request.urlretrieve(row['Url'], row['Location']+".xlsx")

In [159]:
# Step 2 – Streamlining the downloaded files
df = pd.read_excel('output/Mordor.xlsx')
df.head()
# =INDEX($I$1:$R$1, MATCH("OK", I2:R2)) and remove unnesnsary columns
    

,receipt_product_id,product_id,receipt_id,is_return,quantity,location,date,payment_method,sales_rep
0,dcd1c93d-2691-4c10-b0d6-4eaad1b71ebb,PR4938,11,True,1.0,Mordor,2014-04-21 23:38:04,American Express,Gilderoy Lockhart
1,06510aeb-125d-45db-b7c3-8f08b1636ddc,PR5625,11,True,2.0,Mordor,2014-04-21 23:38:04,American Express,Gilderoy Lockhart
2,4b8b4467-d40d-4bd4-a3da-461a9127e443,PR6601,11,True,1.0,Mordor,2014-04-21 23:38:04,American Express,Gilderoy Lockhart
3,f38b4f7d-17af-4c12-af8c-5622e8deb7b7,PR6212,11,True,7.0,Mordor,2014-04-21 23:38:04,American Express,Gilderoy Lockhart
4,d13adb31-3452-47e4-847a-0ae824c4bc3c,PR2186,11,True,4.0,Mordor,2014-04-21 23:38:04,American Express,Gilderoy Lockhart


In [3]:
# Step 3 – Loading everything into a DataFrame

import glob
#glob.glob returns a list of all files in this particular folder

# Create list of DataFrames for each city w list comprehension
# Loop over the files in the output folder and create a DataFrame for each file and add it to a list.
DF_list = [pd.read_excel(city) for city in glob.glob('output/*')] 


In [79]:
# Step 4 – Fetching data from MySql

#You may need to install PyMysql to be able to run this
from sqlalchemy import create_engine
host = "mysql-1.cda.hhs.se"
username = "24996"
password = "toystorey"
schema = "ToyStoreySales"
# format replaces {} in the string with the respective inputs to the function
connection_string = "mysql+pymysql://{}:{}@{}/{}".format(username, password, host,
schema)
# this creates a connection from the string we just created
connection = create_engine(connection_string)
# use the connection to run SQL query and store it into the DataFrame variable df

df_RP_R = pd.read_sql("SELECT * FROM ReceiptProduct RP LEFT JOIN Receipt R ON RP.receipt_id = R.receipt_id", con=connection)
df_RP = pd.read_sql("SELECT * FROM ReceiptProduct RP", con=connection)
df_R = pd.read_sql("SELECT * FROM Receipt", con=connection)
df_P = pd.read_sql("SELECT * FROM Product", con=connection)

In [80]:
# Step 5 – Processing the data
# Add location
df_RP_R["location"] = "internet"
df_RP_R["sales_rep"] = "internet"
# remove discount code
# remove home delivery
# remove customer id
del df_RP_R["discount_code"]
del df_RP_R["home_delivery"]
del df_RP_R["customer_id"]

In [83]:
# Hacky way to delete one reciept_id column. Because we have two columns with same names this was one way to do it.
df_RP_R = df_RP_R.iloc[:, [j for j, c in enumerate(df_RP_R.columns) if j != 1]]
df_RP_R.head()

,receipt_product_id,product_id,quantity,is_return,receipt_id,date,payment_method,location,sales_rep
0,000106ca-430c-4c40-b63e-65dce3be90cc,PR1158,1,0,I-19708,2020-12-04 13:31:20,Master Card,internet,internet
1,0001b70a-eafc-4d71-a9e2-645685c542f9,PR8117,5,0,I-393,2018-05-25 00:47:15,Klarna,internet,internet
2,0001fb36-4f1a-42e4-9141-b0fb74ac09bf,PR4771,1,0,I-18893,2020-11-20 02:41:02,Master Card,internet,internet
3,0002aaa2-31c6-41cf-9305-d8db9e550a48,PR4920,3,0,I-14386,2020-07-20 20:59:03,American Express,internet,internet
4,0004c30a-5f85-42ba-a005-6d4441c670f0,PR4259,1,0,I-8936,2019-11-18 01:16:24,Visa,internet,internet


In [89]:
# Step 5 – Processing the data
# There is a problem with reciept IDs from Internet. Maybe fix that, its not a problem though.
# Add all into one dataframe:
df = df_RP_R
for d in DF_list:
    df = pd.concat([df, d])
df.head()

,receipt_product_id,product_id,quantity,is_return,receipt_id,date,payment_method,location,sales_rep
0,000106ca-430c-4c40-b63e-65dce3be90cc,PR1158,1.0,0,I-19708,2020-12-04 13:31:20,Master Card,internet,internet
1,0001b70a-eafc-4d71-a9e2-645685c542f9,PR8117,5.0,0,I-393,2018-05-25 00:47:15,Klarna,internet,internet
2,0001fb36-4f1a-42e4-9141-b0fb74ac09bf,PR4771,1.0,0,I-18893,2020-11-20 02:41:02,Master Card,internet,internet
3,0002aaa2-31c6-41cf-9305-d8db9e550a48,PR4920,3.0,0,I-14386,2020-07-20 20:59:03,American Express,internet,internet
4,0004c30a-5f85-42ba-a005-6d4441c670f0,PR4259,1.0,0,I-8936,2019-11-18 01:16:24,Visa,internet,internet


In [90]:
# Step 6 – Finding frauds
# Find the sales reps
output = df.groupby(['sales_rep']).sum().reset_index()
output["return_rate"] = (output['is_return'] / output['quantity']).round(2)
output.sort_values(by="return_rate", ascending=False, inplace=True)
print(output)

             sales_rep  quantity  is_return  return_rate
7    Gilderoy Lockhart   15774.0        883         0.06
213      livia månsson     128.0          6         0.05
168    joel gunnarsson     502.0         13         0.03
300    omar henriksson     832.0         22         0.03
0            Beelzebub    1406.0         22         0.02
..                 ...       ...        ...          ...
238    malte nordström    1238.0         15         0.01
191       leah pålsson    1808.0         19         0.01
241    maryam eriksson     208.0          1         0.00
23      alicia sjöberg     130.0          0         0.00
281     nils johansson   70577.0          2         0.00

[382 rows x 4 columns]


In [56]:
# Get all receipts from Gilderoy Lockhart
df_gilderoy = df.loc[df['sales_rep'] == "Gilderoy Lockhart"]
df_gilderoy.head()

,receipt_product_id,product_id,quantity,is_return,date,payment_method,location,sales_rep,receipt_id
0,dcd1c93d-2691-4c10-b0d6-4eaad1b71ebb,PR4938,1.0,1,2014-04-21 23:38:04,American Express,Mordor,Gilderoy Lockhart,11.0
1,06510aeb-125d-45db-b7c3-8f08b1636ddc,PR5625,2.0,1,2014-04-21 23:38:04,American Express,Mordor,Gilderoy Lockhart,11.0
2,4b8b4467-d40d-4bd4-a3da-461a9127e443,PR6601,1.0,1,2014-04-21 23:38:04,American Express,Mordor,Gilderoy Lockhart,11.0
3,f38b4f7d-17af-4c12-af8c-5622e8deb7b7,PR6212,7.0,1,2014-04-21 23:38:04,American Express,Mordor,Gilderoy Lockhart,11.0
4,d13adb31-3452-47e4-847a-0ae824c4bc3c,PR2186,4.0,1,2014-04-21 23:38:04,American Express,Mordor,Gilderoy Lockhart,11.0


In [60]:
#Get total revenue
df_gilderoy = pd.merge(df_gilderoy, df_P, on="product_id", how="left")
df_gilderoy["returned_revenue"] = df_gilderoy['quantity']*df_gilderoy['price']**df_gilderoy['is_return']
print("Gilderoy has returned items of total revenue: " + str(df_gilderoy['returned_revenue'].sum()))


Gilderoy has returned items of total revenue: 24996.0


In [91]:
# Step 7 – Store data for future analysis 
df = df.loc[df['is_return'] != 1]
df = pd.merge(df, df_P, on="product_id", how="left")
df["profit"] = df['quantity']*(df['price']-df['cost'])

,receipt_product_id,product_id,quantity,is_return,receipt_id,date,payment_method,location,sales_rep,product_name,manufacturer,category,price,cost,profit
0,000106ca-430c-4c40-b63e-65dce3be90cc,PR1158,1.0,0,I-19708,2020-12-04 13:31:20,Master Card,internet,internet,Oxford Diecast Blue Land Rover 109 inch 76LAN1...,Oxford Diecast,Die-Cast & Toy Vehicles > Toy Vehicles & Acces...,6.75,3.44,3.31
1,0001b70a-eafc-4d71-a9e2-645685c542f9,PR8117,5.0,0,I-393,2018-05-25 00:47:15,Klarna,internet,internet,Bruder 02052 John Deere 6920 with Frontloader,Bruder,Die-Cast & Toy Vehicles > Toy Vehicles & Acces...,22.50,18.45,20.25
2,0001fb36-4f1a-42e4-9141-b0fb74ac09bf,PR4771,1.0,0,I-18893,2020-11-20 02:41:02,Master Card,internet,internet,Meccano Gears of War C.O.G King Raven,Meccano,None,23.99,15.59,8.40
3,0002aaa2-31c6-41cf-9305-d8db9e550a48,PR4920,3.0,0,I-14386,2020-07-20 20:59:03,American Express,internet,internet,BRIO BRI-33576 Rail Light and Sound Fire Engine,Brio,Die-Cast & Toy Vehicles > Toy Trains & Accesso...,12.85,10.41,7.32
4,0004c30a-5f85-42ba-a005-6d4441c670f0,PR4259,1.0,0,I-8936,2019-11-18 01:16:24,Visa,internet,internet,Disney Frozen Elsa Winter Soft Toy Doll,Disney,Characters & Brands > Disney > Toys,39.70,30.57,9.13


In [97]:
# Store location, date and profit into excel.
output = df.groupby(['location', "date"]).sum().reset_index()[["location", "date", "profit"]]
output.to_excel("aggergated-finacial-data.xlsx") 
output.head()

,location,date,profit
0,Alingsås,2014-04-21 08:45:34,8.04
1,Alingsås,2014-04-27 08:49:02,172.34
2,Alingsås,2014-04-28 07:50:54,37.58
3,Alingsås,2014-05-03 15:09:18,17.76
4,Alingsås,2014-05-06 18:53:40,56.39
